In [ ]:
import os
os.environ['PYSPARK_PYTHON'] = '/opt/conda/bin/python'
os.environ['PYSPARK_DRIVER_PYTHON'] = '/opt/conda/bin/python'

In [ ]:
import datalabframework as dlf
import pyspark.sql.functions as F
import pyspark.sql.types as T
import yaml
import pandas as pd
from datetime import datetime, timedelta
from util_report.util_reports import *
from util_report.util_render import *
from util_report.helpers import *

In [ ]:
pd.set_option('display.max_columns', 30)

### Setup engine

In [ ]:
dlf.project.load('default')
engine = dlf.project.engine()
spark = engine.context() 
md = dlf.project.metadata()

### Setup date

In [ ]:
# md = dlf.project.metadata()
# report_date_str = md['variables']['report_date']
report_date_str = "2019-06-05"
format_str= "%Y-%m-%d"
report_date = datetime.strptime(report_date_str, format_str)
cube_date = report_date - timedelta(days=1)
first_of_last_month = (report_date.replace(day=1) - timedelta(days=1)).replace(day=1).replace(hour=0)
lastweek = (report_date - timedelta(days=7))
this_month = (report_date - timedelta(days=1)).month
this_year = (report_date - timedelta(days=1)).year

### Get config

In [ ]:
config = yaml.load(open('config/config.yml'))
config_email = config['email'][md['variables']['email_env']]
config_struct = config

In [ ]:
pd_df = None

### Create html table from pandas dataframe

In [ ]:
config_table = config_struct['table']
report_jinja = get_data_before_render(config_table, pd_df, report_date)

### Accumulate all html tables

In [ ]:
content = ""
content += render_header(config_struct['header'])
content += render_table(report_jinja)

### Render file html

In [ ]:
write_to_file_html('report.html', content)

### Send email

In [ ]:
send_mail_with_cc(config_email, report_date, content)